In [1]:
from bitarray import bitarray
import struct
import os

In [2]:
def openFile(path):
    """Otwiera plik i zwraca jego zawartosc"""
    with open(path, "rb") as f:
        return f.read()

def decToBin(n):
    """Zamienia liczbe dziesietna na binarna"""
    if n == 0:
        return "0"
    elif n == 1:
        return "1"
    else:
        return decToBin(n // 2) + str(n % 2)

def toBits(n):
    """Zamienia liczbe na binarna i dodaje zera na poczatek"""
    bits = decToBin(n)
    #kod ma 6 bitow
    while len(bits) < 6:
        bits = "0" + bits
    return bits

In [3]:
def encode(text, maxSize= None):
    """Kodowanie LZW"""
    dictionary = {bytes([i]): i for i in range(256)}
    dict_size = 256

    p = b""          # bieżący „pattern”
    encoded = []     # zakodowane dane

    for byte in text:
        c = bytes([byte])
        pc = p + c
        if pc in dictionary: # jeśli jest w słowniku
            p = pc
        else:
            encoded.append(dictionary[p])
            # jeśli nie ma
            if maxSize is None or dict_size < maxSize:
                dictionary[pc] = dict_size
                dict_size += 1
            p = c

    #ostatni pattern
    if p:
        encoded.append(dictionary[p])

    return encoded

In [4]:
def decode(text, maxSize=None):
    """Dekodowanie LZW"""
    dictionary = {i: bytes([i]) for i in range(256)}
    dict_size = 256

    # Pierwszy kod
    iter_codes = iter(text)
    first_code = next(iter_codes)
    w = dictionary[first_code]
    result = bytearray(w)

    # Reszta
    for k in iter_codes:
        if k in dictionary:
            entry = dictionary[k]
        elif k == dict_size:
            # przypadek specialny LZW
            entry = w + w[:1]
        else:
            raise ValueError("Nieprawidłowy kod w dekodowaniu LZW")
        
        result.extend(entry)
        # dodaj nowy wpis do słownika
        if maxSize is None or dict_size < maxSize:
            dictionary[dict_size] = w + entry[:1]
            dict_size += 1
        w = entry

    return bytes(result)

In [5]:
def save(text, path):
    """Zapisuje zakodowane dane do pliku"""
    with open(path, 'wb') as f:
        for code in text:
            f.write(struct.pack('>I', code))


def load(path):
    """Wczytuje plik i zwraca jego zawartosc"""
    codes = []
    with open(path, 'rb') as f:
        while True:
            bytes_read = f.read(4)
            if not bytes_read:
                break
            codes.append(struct.unpack('>I', bytes_read)[0])
    return codes

In [6]:
pliki = ['dane/norm_wiki_sample.txt', 'dane/wiki_sample.txt', 'dane/lena.bmp']
limity = [None, 2**12, 2**18]

for filepath in pliki:
        data = openFile(filepath)
        orig_size = len(data)
        for lim in limity:
            codes = encode(data, maxSize=lim)
            out_name = os.path.splitext(os.path.basename(filepath))[0]
            limit_str = 'brak' if lim is None else str(lim)
            out_path = f"wyniki/{out_name}_{limit_str}.bin"
            save(codes, out_path)
            comp_size = os.path.getsize(out_path)

            # weryfikacja poprawności dekodowania
            loaded_codes = load(out_path)
            recovered = decode(loaded_codes, maxSize=lim)
            if data != recovered:
                print(f"Blad dekodowania w pliku {out_path}")
                continue
            
            # Wyświetlenie szczegółowych informacji o kompresji
            file_name = os.path.basename(filepath)
            print("\n================ Kompresja pliku ================")
            print(f"Plik:               {file_name}")
            print(f"Limit słownika:     {lim}")
            print(f"Rozmiar oryginału:  {orig_size} bajtów")
            print(f"Rozmiar skompresowany: {comp_size} bajtów")
            print("================================================")



================ Kompresja pliku ================
Plik:               norm_wiki_sample.txt
Limit słownika:     None
Rozmiar oryginału:  10788941 bajtów
Rozmiar skompresowany: 6326240 bajtów

================ Kompresja pliku ================
Plik:               norm_wiki_sample.txt
Limit słownika:     4096
Rozmiar oryginału:  10788941 bajtów
Rozmiar skompresowany: 14540432 bajtów

================ Kompresja pliku ================
Plik:               norm_wiki_sample.txt
Limit słownika:     262144
Rozmiar oryginału:  10788941 bajtów
Rozmiar skompresowany: 7345816 bajtów

================ Kompresja pliku ================
Plik:               wiki_sample.txt
Limit słownika:     None
Rozmiar oryginału:  11909016 bajtów
Rozmiar skompresowany: 7177024 bajtów

================ Kompresja pliku ================
Plik:               wiki_sample.txt
Limit słownika:     4096
Rozmiar oryginału:  11909016 bajtów
Rozmiar skompresowany: 17755648 bajtów

================ Kompresja pliku ================
